# Text Normalization

We use lots of variations for same words for examples user might express love in different ways `[lovely, luv, loveee, ...]` all these are the essentially the same to a machine learning model that tries to find out if a review is positive or negative.

So in this notebook we will discuss some methods to reduce text variations.

# Lemmatization vs Stemming

The key concept here is that stemming sometime destroy the word unlike lemmatization where we keep the meaning.


# Stemming

Stemming reduce the text by a set of pre-defined rules like removing `ing` from verbs

In [1]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

plurals = ['caresses', 'flies', 'dies', 'mules', 'denied',
           'died', 'agreed', 'owned', 'humbled', 'sized',
           'meeting', 'stating', 'siezing', 'itemization',
           'sensational', 'traditional', 'reference', 'colonizer',
           'plotted']

singles = [stemmer.stem(plural) for plural in plurals]
print(singles)

['caress', 'fli', 'die', 'mule', 'deni', 'die', 'agre', 'own', 'humbl', 'size', 'meet', 'state', 'siez', 'item', 'sensat', 'tradit', 'refer', 'colon', 'plot']


# Lemmatization

Lemmatizer reduce the word by looking it up in the `WordNet` where it tries to find the root of the word for example `rocks` -> `rock`

In [2]:
# import these modules
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# (plural to singular form)
print("rocks    :", lemmatizer.lemmatize("rocks"))

# a denotes adje (irregular plural form)
print("corpora  :", lemmatizer.lemmatize("corpora"))
# active in "pos"
print("-----better-----")
print('Noun : ',lemmatizer.lemmatize("better"))         # Output: "better" (default noun assumption)
print('Adj  : ',lemmatizer.lemmatize("better", pos="a"))  # Output: "good" (adjective form)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mmaba\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


rocks    : rock
corpora  : corpus
-----better-----
Noun :  better
Adj  :  good


In [3]:
import os

for dirname, _, filenames in os.walk('./All Hadith Books'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./All Hadith Books\Maliks Muwatta Without_Tashkel.csv
./All Hadith Books\Maliks Muwatta.csv
./All Hadith Books\Musnad Ahmad ibn Hanbal Without_Tashkel.csv
./All Hadith Books\Musnad Ahmad ibn Hanbal.csv
./All Hadith Books\Sahih Bukhari Without_Tashkel.csv
./All Hadith Books\Sahih Bukhari.csv
./All Hadith Books\Sahih Muslim.csv
./All Hadith Books\Sahih Muslime Without_Tashkel.csv
./All Hadith Books\Sunan Abu Dawud Without_Tashkel.csv
./All Hadith Books\Sunan Abu Dawud.csv
./All Hadith Books\Sunan al Darami Without_Tashkel.csv
./All Hadith Books\Sunan al Darami.csv
./All Hadith Books\Sunan al Tirmidhi Without_Tashkel.csv
./All Hadith Books\Sunan al Tirmidhi.csv
./All Hadith Books\Sunan al-Nasai Without_Tashkel.csv
./All Hadith Books\Sunan al-Nasai.csv
./All Hadith Books\Sunan Ibn Maja Without_Tashkel.csv
./All Hadith Books\Sunan Ibn Maja.csv


In [4]:

import re


def normalize(sentence):
    '''
    Argument:
        string of words
    return:
        string of words but standardize the words
    '''
    sentence = re.sub("[إأآا]", "ا", sentence)
    sentence = re.sub("ى", "ي", sentence)
    sentence = re.sub("ؤ", "ء", sentence)
    sentence = re.sub("ئ", "ء", sentence)
    sentence = re.sub("ة", "ه", sentence)
    sentence = re.sub("گ", "ك", sentence)
    return sentence


In [5]:
from nltk.corpus import stopwords

stopwords = stopwords.words('arabic')

In [6]:
from nltk import word_tokenize


def removing_ar_stopwords(text):
    """
        Here we remove all Arabic stop words
        
    """
    original_words = []
    words = word_tokenize(text)  # it works on one hadith not list
    for word in words:
        if word not in stopwords:
            original_words.append(word)
    filtered_sentence = " ".join(original_words)
    return filtered_sentence


In [7]:

from nltk import ISRIStemmer


def stemming_1(text):
    """
        This is first functoin for stemming and it's looks not good accurac, NLTK by ISRIStemmer.
    """
    st = ISRIStemmer()
    stemmend_words = []
    words = word_tokenize(text)
    for word in words:
        stemmend_words.append(st.stem(word))
    stemmed_sentence = " ".join(stemmend_words)
    return stemmed_sentence


def stemming_2(text):
    """
        This is Second functoin for stemming and it's looks good results, with built in library called Tashaphyne.
        The documentation here ==> https://pypi.org/project/Tashaphyne/
    
    """
    from tashaphyne.stemming import ArabicLightStemmer
    ArListem = ArabicLightStemmer()
    words = word_tokenize(text)
    new_list = []
    for word in words:
        stem = ArListem.light_stem(word)
        stem = ArListem.get_stem()
        new_list.append(stem)

        hadith_sentence_with_stemming = " ".join(new_list)

    return hadith_sentence_with_stemming


* **`Stemming`** : algorithms work by cutting off the end or the beginning of the word, taking into account a list of common prefixes and suffixes that can be found in an inflected word.
* **`Lemmatization`** : takes into consideration the morphological analysis of the words. 

**Lemmatization is typically seen as much more informative than simple stemming, because stem may not be an actual word whereas lemma is an actual language word.**

#### Upove some functions for like:
* stemming_1 by `ISRIStemmer from NLTK`.
* stemming_2 by `Tashaphyne` is an Arabic light stemmer(removing prefixes and suffixes) and give all possible segmentations. 
* lemmatization by [Farasa API](https://alt.qcri.org/farasa/)

**By the experimental: lemmatization by Farasa have a good results.**

In [8]:

import pandas as pd

data_1 = pd.read_csv(dirname + '/Maliks Muwatta Without_Tashkel.csv')
data_1.head()

,Maliks Muwatta Without_Tashkel
0,قال حدثني الليثي عن مالك بن أنس عن ابن شهاب أن...
1,و حدثني يحيى عن مالك عن زيد بن أسلم عن عطاء بن...
2,و حدثني يحيى عن مالك عن يحيى بن سعيد عن عمرة ب...
3,و حدثني عن مالك عن زيد بن أسلم عن عطاء بن يسار...
4,و حدثني عن مالك عن نافع مولى عبد الله بن عمر أ...


In [9]:
all_hadiths_1 = []

for hadith in data_1['Maliks Muwatta Without_Tashkel']:
    all_hadiths_1.append(hadith)


In [10]:
all_hadiths_1[0:1]

['قال حدثني الليثي عن مالك بن أنس عن ابن شهاب أن عمر بن عبد العزيز أخر الصلاة يوما فدخل عليه عروة بن الزبير فأخبره أن المغيرة بن شعبة أخر الصلاة يوما وهو بالكوفة فدخل عليه أبو مسعود الأنصاري فقال ما هذا يا مغيرة أليس قد علمت أن جبريل نزل فصلى فصلى رسول الله صلى الله عليه وسلم ثم صلى فصلى رسول الله صلى الله عليه وسلم ثم صلى فصلى رسول الله صلى الله عليه وسلم ثم صلى فصلى رسول الله صلى الله عليه وسلم ثم صلى فصلى رسول الله صلى الله عليه وسلم ثم قال بهذا أمرت فقال عمر بن عبد العزيز اعلم ما تحدث به يا عروة أو إن جبريل هو الذي أقام لرسول الله صلى الله عليه وسلم وقت الصلاة قال عروة كذلك كان بشير بن أبي مسعود الأنصاري يحدث عن أبيه قال عروة ولقد حدثتني عائشة زوج النبي صلى الله عليه وسلم أن رسول الله صلى الله عليه وسلم كان يصلي العصر والشمس في حجرتها قبل أن تظهر.']

In [11]:
##%%time

# Maliks Muwatta
cleared_Hadith_1 = []  # Removing stopwords

for hadith in all_hadiths_1:
    cleared_Hadith_1.append(removing_ar_stopwords(hadith))  # Removing stopwords

len(cleared_Hadith_1)
cleared_Hadith_1[0:1]

['قال حدثني الليثي مالك بن أنس ابن شهاب عمر بن عبد العزيز أخر الصلاة يوما فدخل عروة بن الزبير فأخبره المغيرة بن شعبة أخر الصلاة يوما بالكوفة فدخل مسعود الأنصاري فقال مغيرة أليس علمت جبريل نزل فصلى فصلى رسول الله صلى الله وسلم صلى فصلى رسول الله صلى الله وسلم صلى فصلى رسول الله صلى الله وسلم صلى فصلى رسول الله صلى الله وسلم صلى فصلى رسول الله صلى الله وسلم قال بهذا أمرت فقال عمر بن عبد العزيز اعلم تحدث عروة جبريل أقام لرسول الله صلى الله وسلم وقت الصلاة قال عروة بشير بن أبي مسعود الأنصاري يحدث أبيه قال عروة ولقد حدثتني عائشة زوج النبي صلى الله وسلم رسول الله صلى الله وسلم يصلي العصر والشمس حجرتها تظهر .']

In [12]:
cleared_Hadith_1_2 = []  # Normalization

for hadith in cleared_Hadith_1:
    cleared_Hadith_1_2.append(normalize(hadith))  # Normalization

len(cleared_Hadith_1_2)
cleared_Hadith_1_2[0:1]

['قال حدثني الليثي مالك بن انس ابن شهاب عمر بن عبد العزيز اخر الصلاه يوما فدخل عروه بن الزبير فاخبره المغيره بن شعبه اخر الصلاه يوما بالكوفه فدخل مسعود الانصاري فقال مغيره اليس علمت جبريل نزل فصلي فصلي رسول الله صلي الله وسلم صلي فصلي رسول الله صلي الله وسلم صلي فصلي رسول الله صلي الله وسلم صلي فصلي رسول الله صلي الله وسلم صلي فصلي رسول الله صلي الله وسلم قال بهذا امرت فقال عمر بن عبد العزيز اعلم تحدث عروه جبريل اقام لرسول الله صلي الله وسلم وقت الصلاه قال عروه بشير بن ابي مسعود الانصاري يحدث ابيه قال عروه ولقد حدثتني عاءشه زوج النبي صلي الله وسلم رسول الله صلي الله وسلم يصلي العصر والشمس حجرتها تظهر .']

In [13]:
cleared_Hadith_1_2_3 = []

for hadith in cleared_Hadith_1_2:
    cleared_Hadith_1_2_3.append(stemming_1(hadith))  #ISRIStemmer

print('The size of data:')
len(cleared_Hadith_1_2_3)
cleared_Hadith_1_2_3[0:1]

The size of data:


['قال حدث ليث الك بن انس ابن شهب عمر بن عبد عزز اخر صله يوم دخل عره بن زبر خبر غير بن شعب اخر صله يوم كوف دخل سعد صار فقل غير الس علم جبريل نزل فصل فصل رسل الل صلي الل سلم صلي فصل رسل الل صلي الل سلم صلي فصل رسل الل صلي الل سلم صلي فصل رسل الل صلي الل سلم صلي فصل رسل الل صلي الل سلم قال بهذا امر فقل عمر بن عبد عزز علم حدث عره جبريل اقم رسل الل صلي الل سلم وقت صله قال عره بشر بن ابي سعد صار حدث ابه قال عره لقد حدث عءش زوج نبي صلي الل سلم رسل الل صلي الل سلم يصل عصر شمس حجر ظهر .']

In [14]:
cleared_Hadith_1_2_3_4 = []

for hadith in cleared_Hadith_1_2:
    cleared_Hadith_1_2_3_4.append(stemming_2(hadith))  #ArabicLightStemmer

print('The size of data:')
len(cleared_Hadith_1_2_3_4)
cleared_Hadith_1_2_3_4[0:1]

The size of data:


['قال حدث ليث مال بن نس بن شهاب عمر بن عبد عزيز خر صلاه وم دخل عر بن زبير خبر مغيره بن شعب خر صلاه وم كوفه دخل مسعود انصار قال مغير يس علم جبريل زل صل صل رسول له صل له سلم صل صل رسول له صل له سلم صل صل رسول له صل له سلم صل صل رسول له صل له سلم صل صل رسول له صل له سلم قال هذا مرت قال عمر بن عبد عزيز علم حدث عر جبريل قام رسول له صل له سلم قت صلاه قال عر شير بن بي مسعود انصار حدث بي قال عر قد حدث عاءش زوج نب صل له سلم رسول له صل له سلم صلي عصر شمس حجر ظهر .']

In [15]:
# make it as a DataFram 
Maliks_Muwatta_preprosessing_1 = pd.DataFrame(cleared_Hadith_1_2_3, columns=['Maliks_Muwatta_Preprosessing_Cleaned'])
Maliks_Muwatta_preprosessing_1.head()

,Maliks_Muwatta_Preprosessing_Cleaned
0,قال حدث ليث الك بن انس ابن شهب عمر بن عبد عزز ...
1,حدث يحي الك زيد بن سلم عطء بن يسر انه قال جاء ...
2,حدث يحي الك يحي بن سعد عمر بنت عبد رحم عءش زوج...
3,حدث الك زيد بن سلم عطء بن يسر وعن بسر بن سعد و...
4,حدث الك نفع ولي عبد الل بن عمر عمر بن خطب كتب ...
